*   Author : Ziang Xu
*   Student number : 180104048
*   Code : Vgg-16 model construction, training and testing.
*   Grad-CAM Visualization.
      

In [0]:
# Google Colaboratory carry Google drive.
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
# Enter the classification task directory.
!mkdir -p drive
!google-drive-ocamlfuse drive
import os
os.chdir("drive/ML/classification/")
!ls

In [0]:
'''
Visual task requires Keras, a fixed version of tensorflow, 
but colab is automatically reset every day. So we need to reconfigure the version.
'''
!pip show keras
!pip show tensorflow
!pip show keras-vis

In [0]:
!pip uninstall keras
!pip uninstall tensorflow
!pip uninstall keras-vis

In [0]:
!pip install tensorflow==1.12
!pip install keras==2.2.4
!pip install git+https://github.com/raghakot/keras-vis.git

In [0]:
# Read dataset
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
images=[]
labels=[]
def read_image(imageName):
    im = Image.open(imageName).convert('RGB')
    data = np.array(im)
    return data

In [0]:
# Read there are several folders in the dataset.
text = os.listdir('./resize_data')
# Save the name of the image in the folder and its corresponding folder to the corresponding list.
for textPath in text:
    for fn in os.listdir(os.path.join('resize_data', textPath)):
        if fn.endswith('.jpg'):
            fd = os.path.join('./resize_data', textPath, fn)
            images.append(read_image(fd))
            labels.append(textPath)
            
X = np.array(images)
y = np.array(list(map(int, labels))) 

In [0]:
# Split dataset to 0.8 for training and 0.2 for testing.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=30)

In [0]:
# [0],dyed-lifted-polyps
# [1],dyed-resection-margins
# [2],esophagitis
# [3],normal-cecum
# [4],normal-pylorus
# [5],normal-z-line
# [6],polyps
# [7],ulcerative-colitis
plt.imshow(X_train[0])

In [0]:
# Normalized dataset
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
from keras.utils import np_utils
Y_train = np_utils.to_categorical(y_train, 8)
Y_test = np_utils.to_categorical(y_test, 8)

In [0]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

In [0]:
# Fine-tuning technique is used to build model.
from keras.applications.vgg16 import VGG16

from keras.layers import Input
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import Model
from keras.optimizers import SGD
# Load model pre-training weights.
model_vgg = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
for layer in model_vgg.layers:
    layer.trainable = False
# Remove the fully connected layer and join the fully connected layer that belongs to us.   
model = Flatten(name='flatten')(model_vgg.output)
model = Dense(4096, activation='relu', name='fc1')(model)
model = Dense(4096, activation='relu', name='fc2')(model)
model = Dropout(0.5)(model)
model = Dense(8, activation='softmax')(model)
model_vgg16 = Model(inputs=model_vgg.input, outputs=model, name='vgg16')



model_vgg16.summary()

In [0]:
# Set the callback to calculate the accuracy of each training.
import numpy
from keras.callbacks import Callback
from sklearn.metrics import f1_score, precision_score, recall_score


class Metrics(Callback):
    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []

    def on_epoch_end(self, epoch, logs={}):
        val_predict = (numpy.asarray(self.model.predict(
            self.validation_data[0]))).round()
        val_targ = self.validation_data[1]
        _val_f1 = f1_score(val_targ, val_predict,average='macro')
        _val_recall = recall_score(val_targ, val_predict,average='macro')
        _val_precision = precision_score(val_targ, val_predict,average='macro')
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        print(_val_f1,_val_precision,_val_recall)
        return
metrics = Metrics()

In [0]:
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def fbeta_score(y_true, y_pred, beta=1):
    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')

    # If there are no true positives, fix the F score at 0 like sklearn.
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score

def fmeasure(y_true, y_pred):
    return fbeta_score(y_true, y_pred, beta=1)

In [0]:
# Customize the optimizer and compile the model.
import keras
opt_sgd = keras.optimizers.SGD(
                lr=1e-3
                , decay=1e-9
                , momentum=0.9
                , nesterov=False
            )
model.compile(
    loss='categorical_crossentropy'
    , optimizer=opt_sgd
    , metrics=[precision,recall,fbeta_score]
)

In [0]:
from keras.callbacks import ModelCheckpoint
model_checkpoint = ModelCheckpoint('vgg_16.h5', monitor='loss', verbose=1, save_best_only=True)

In [0]:
# Training
print('Training begin')
history=model.fit(X_train, Y_train, batch_size=50, epochs=1,
          validation_data=(X_test, Y_test),callbacks=[metrics])

In [0]:
# Draw a graph of accuracy and loss functions.
print(history.history.keys())

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
# Model saving and loading.
model_vgg16.save('model1.h5')
from keras.models import load_model
model=load_model('vgg-Bilinearnet.h5')


In [0]:
from keras.utils import plot_model
plot_model(model,to_file='WRN.png',show_shapes=True)

In [0]:
# Model evaluation
scores = model.evaluate(X_test, Y_test)
print('Test score:', scores[0])
print('Test accuracy:', scores[1])

In [0]:
# Other evaluation methods
from sklearn import metrics
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [0]:
y_pred=model.predict(X_test)
y_true=Y_test
y_pred=np.argmax(y_pred, axis=1)
y_true=np.argmax(y_true, axis=1)

In [0]:
metrics.precision_score(y_true, y_pred,average='weighted')

In [0]:
accuracy_score(y_true, y_pred)

In [0]:
metrics.recall_score(y_true, y_pred,average='weighted')

In [0]:
metrics.f1_score(y_true, y_pred,average='macro')

In [0]:
conf_arr=confusion_matrix(y_true, y_pred)

In [0]:
target_names = ['0', '1', '2','3','4','5','6','7']
print(classification_report(y_true, y_pred, target_names=target_names))

In [0]:
!pip install scikit-plot
import scikitplot as skplt
plot = skplt.metrics.plot_confusion_matrix(y_true, y_pred)

In [0]:
# Model visualization
from keras.applications import VGG16
from vis.utils import utils
from keras import activations

from vis.utils import utils
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (18,6)

layer_idx = utils.find_layer_idx(model, 'dense_1')

# Swap softmax with linear
model.layers[layer_idx].activation = activations.linear
model = utils.apply_modifications(model)

img1 = utils.load_img('image/1.jpg', target_size=(224, 224))
img2 = utils.load_img('image/2.jpg', target_size=(224, 224))
img3 = utils.load_img('image/3.jpg', target_size=(224, 224))
img4 = utils.load_img('image/4.jpg', target_size=(224, 224))
img5 = utils.load_img('image/5.jpg', target_size=(224, 224))

f,ax=plt.subplots(1,5)
ax[0].imshow(img1)
ax[1].imshow(img2)
ax[2].imshow(img3)
ax[3].imshow(img4)
ax[4].imshow(img5)


In [0]:
img6 = utils.load_img('image/6.jpg', target_size=(224, 224))
img7 = utils.load_img('image/7.jpg', target_size=(224, 224))
img8 = utils.load_img('image/8.jpg', target_size=(224, 224))
img9 = utils.load_img('image/9.jpg', target_size=(224, 224))
img10 = utils.load_img('image/10.jpg', target_size=(224, 224))

f,ax=plt.subplots(1,5)
ax[0].imshow(img6)
ax[1].imshow(img7)
ax[2].imshow(img8)
ax[3].imshow(img9)
ax[4].imshow(img10)

In [0]:
from vis.visualization import visualize_saliency, overlay
from vis.utils import utils
from keras import activations

# Utility to search for layer index by name. 
# Alternatively we can specify this as -1 since it corresponds to the last layer.
layer_idx = utils.find_layer_idx(model, 'dense_1')

f, ax = plt.subplots(1,2)
for i, img in enumerate([img1, img2]): 
    # 0 is the imagenet index corresponding to `dyed-lifted-polyps`
    grads = visualize_saliency(model, layer_idx, filter_indices=0, seed_input=img)
    
    # visualize grads as heatmap
    ax[i].imshow(grads, cmap='jet')

In [0]:
for modifier in ['guided', 'relu']:
    plt.figure()
    f, ax = plt.subplots(1, 2)
    plt.suptitle(modifier)
    for i, img in enumerate([img1, img2]):   
        # 0 is the imagenet index corresponding to `dyed-lifted-polyps`
        grads = visualize_saliency(model, layer_idx, filter_indices=0, 
                                   seed_input=img, backprop_modifier=modifier)
        # Lets overlay the heatmap onto original image.    
        ax[i].imshow(grads, cmap='jet')

In [0]:
import numpy as np
import matplotlib.cm as cm
from vis.visualization import visualize_cam

for modifier in [None, 'guided', 'relu']:
    plt.figure()
    f, ax = plt.subplots(1, 2)
    plt.suptitle("vanilla" if modifier is None else modifier)
    for i, img in enumerate([img1, img2]):  
        # 0 is the imagenet index corresponding to `dyed-lifted-polyps`
        grads = visualize_cam(model, layer_idx, filter_indices=0, 
                              seed_input=img, backprop_modifier=modifier)        
        # Lets overlay the heatmap onto original image.    
        jet_heatmap = np.uint8(cm.jet(grads)[..., :3] * 255)
        ax[i].imshow(overlay(jet_heatmap, img))

In [0]:
import numpy as np
import matplotlib.cm as cm
from vis.visualization import visualize_cam

for modifier in [None, 'guided', 'relu']:
    plt.figure()
    f, ax = plt.subplots(1, 2)
    plt.suptitle("vanilla" if modifier is None else modifier)
    for i, img in enumerate([img3, img4]):  
        # 1 is the imagenet index corresponding to `dyed-resection-margins`
        grads = visualize_cam(model, layer_idx, filter_indices=1, 
                              seed_input=img, backprop_modifier=modifier)        
        # Lets overlay the heatmap onto original image.    
        jet_heatmap = np.uint8(cm.jet(grads)[..., :3] * 255)
        ax[i].imshow(overlay(jet_heatmap, img))

In [0]:
import numpy as np
import matplotlib.cm as cm
from vis.visualization import visualize_cam

for modifier in ['guided']:
    plt.figure()
    f, ax = plt.subplots(1, 2)
    plt.suptitle("vanilla" if modifier is None else modifier)
    for i, img in enumerate([img5, img6]):  
        # 2 is the imagenet index corresponding to `esophagitis`
        grads = visualize_cam(model, layer_idx, filter_indices=2, 
                              seed_input=img, backprop_modifier=modifier)        
        # Lets overlay the heatmap onto original image.    
        jet_heatmap = np.uint8(cm.jet(grads)[..., :3] * 255)
        ax[i].imshow(overlay(jet_heatmap, img))

In [0]:
import numpy as np
import matplotlib.cm as cm
from vis.visualization import visualize_cam

for modifier in ['guided']:
    plt.figure()
    f, ax = plt.subplots(1, 2)
    plt.suptitle("vanilla" if modifier is None else modifier)
    for i, img in enumerate([img7, img8]):  
        # 6 is the imagenet index corresponding to `polyps`
        grads = visualize_cam(model, layer_idx, filter_indices=6, 
                              seed_input=img, backprop_modifier=modifier)        
        # Lets overlay the heatmap onto original image.    
        jet_heatmap = np.uint8(cm.jet(grads)[..., :3] * 255)
        ax[i].imshow(overlay(jet_heatmap, img))

In [0]:
import numpy as np
import matplotlib.cm as cm
from vis.visualization import visualize_cam

for modifier in ['guided']:
    plt.figure()
    f, ax = plt.subplots(1,1)
    plt.suptitle(label[y])
    
    # 7 is the imagenet index corresponding to 'ulcerative-colitis'
    grads = visualize_cam(model, layer_idx, filter_indices=7, 
                              seed_input=img9, backprop_modifier=modifier)        
    # Lets overlay the heatmap onto original image.    
    jet_heatmap = np.uint8(cm.jet(grads)[..., :3] * 255)
    ax.imshow(overlay(jet_heatmap, img9))

In [0]:
from keras.preprocessing import image
import numpy as np
from keras.utils.np_utils import *

# Model prediction.
file_path='image/1.jpg' 
img=image.load_img(file_path,target_size=(224,224))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)


y=model.predict(x,verbose=1)

y=y.argmax(axis=-1)

label=['dyed-lifted-polyps','dyed-resection-margins','esophagitis','normal-cecum',
       'normal-pylorus','normal-z-line','polyps','ulcerative-colitis']
label=np.array(label)

print(y,label[y])